# Statistical Language Models – Advanced Smoothing & Evaluation

This notebook explores **advanced techniques** for building better statistical language models.

### Advanced Smoothing Techniques
Moving beyond basic Laplace smoothing to more sophisticated methods:

1. **Good-Turing Smoothing** - Adjusts probabilities based on frequency-of-frequencies
2. **Kneser-Ney Smoothing** - Uses word continuation probability (state-of-the-art for n-gram models)
3. **Comparison** - Understanding when each method performs best

### Model Evaluation
- **Perplexity** - Standard metric for comparing language models
- **Cross-validation** - Testing on real text data
- **In-domain vs Out-of-domain** - Performance on familiar vs unfamiliar text

### Extension to Higher-Order Models
- **Trigram models** - Looking back 2 words instead of 1
- **Trade-offs** - Computational cost vs accuracy

---

📌 **Dataset Resources**
- Project Gutenberg: https://www.gutenberg.org/
- Example text: https://www.gutenberg.org/files/1342/1342-0.txt (Pride and Prejudice)


## 1. Load and Preprocess Text

First, let's set up our preprocessing pipeline and load sample text data.

In [1]:
import re
from collections import Counter, defaultdict

def preprocess(text):
    """Clean and tokenize text"""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text.split()

# Sample text with multiple sentences
sample_text = """
I love natural language processing.
Language models help predict words.
I love learning language models.
"""

tokens = preprocess(sample_text)
print(f"Tokens: {tokens}")
print(f"Total tokens: {len(tokens)}")
print(f"Unique words: {len(set(tokens))}")

Tokens: ['i', 'love', 'natural', 'language', 'processing', 'language', 'models', 'help', 'predict', 'words', 'i', 'love', 'learning', 'language', 'models']
Total tokens: 15
Unique words: 10


## 2. Unigram & Bigram Counts

Build the foundational counts we'll use for all smoothing techniques.

In [2]:
# Count individual words
unigram_counts = Counter(tokens)

# Count word pairs
bigram_counts = defaultdict(int)
for i in range(len(tokens) - 1):
    bigram_counts[(tokens[i], tokens[i+1])] += 1

print("Unigram Counts:")
for word, count in unigram_counts.most_common():
    print(f"  {word}: {count}")

print(f"\nBigram Counts:")
for bigram, count in sorted(bigram_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {bigram}: {count}")

Unigram Counts:
  language: 3
  i: 2
  love: 2
  models: 2
  natural: 1
  processing: 1
  help: 1
  predict: 1
  words: 1
  learning: 1

Bigram Counts:
  ('i', 'love'): 2
  ('language', 'models'): 2
  ('love', 'natural'): 1
  ('natural', 'language'): 1
  ('language', 'processing'): 1
  ('processing', 'language'): 1
  ('models', 'help'): 1
  ('help', 'predict'): 1
  ('predict', 'words'): 1
  ('words', 'i'): 1
  ('love', 'learning'): 1
  ('learning', 'language'): 1


## 3. Laplace (Add-One) Smoothing

**Review from previous notebook:** Laplace smoothing adds 1 to all counts.

**Formula:**  
$$P_{\text{Laplace}}(w_2|w_1) = \frac{\text{count}(w_1, w_2) + 1}{\text{count}(w_1) + V}$$

**Characteristics:**
- Simple and intuitive
- Can over-smooth (gives too much probability to unseen events)
- Works reasonably well for small vocabularies

In [3]:
vocab = set(tokens)
V = len(vocab)

laplace_bigram_probs = {}
for w1 in vocab:
    for w2 in vocab:
        count = bigram_counts.get((w1, w2), 0)
        laplace_bigram_probs[(w1, w2)] = (count + 1) / (unigram_counts[w1] + V)

print(f"Vocabulary size: {V}")
print(f"Total bigram probabilities: {len(laplace_bigram_probs)}")
print("\nSample Laplace probabilities:")
for bigram, prob in list(laplace_bigram_probs.items())[:5]:
    original_count = bigram_counts.get(bigram, 0)
    print(f"  {bigram}: count={original_count}, P={prob:.6f}")

Vocabulary size: 10
Total bigram probabilities: 100

Sample Laplace probabilities:
  ('i', 'i'): count=0, P=0.083333
  ('i', 'predict'): count=0, P=0.083333
  ('i', 'love'): count=2, P=0.250000
  ('i', 'language'): count=0, P=0.083333
  ('i', 'models'): count=0, P=0.083333


## 4. Good-Turing Smoothing

**Good-Turing smoothing** uses the frequency-of-frequencies to estimate probabilities of unseen events.

### Key Idea
- Count how many items appear once, twice, three times, etc.
- Use items seen $(r+1)$ times to estimate probability of items seen $r$ times
- Items never seen get probability based on items seen once

**Formula:**  
$$r^* = \frac{(r+1) \cdot N_{r+1}}{N_r}$$

Where:
- $r$ = observed count
- $N_r$ = number of items with count $r$
- $r^*$ = adjusted count

**Advantages:**
- More principled than Laplace
- Better handling of rare events
- Based on the actual distribution of frequencies

In [4]:
from collections import Counter

def good_turing(counts):
    """
    Apply Good-Turing smoothing to a dictionary of counts.
    Returns adjusted probability for each frequency level.
    """
    # Count frequencies of frequencies
    freq_of_freq = Counter(counts.values())
    N = sum(counts.values())  # Total count
    gt_probs = {}

    for r, Nr in freq_of_freq.items():
        Nr1 = freq_of_freq.get(r + 1, 0)  # Items with frequency r+1
        if Nr1 > 0:
            # Adjust count using Good-Turing formula
            r_star = (r + 1) * (Nr1 / Nr)
            gt_probs[r] = r_star / N
        else:
            # Fallback to simple probability if no r+1 items
            gt_probs[r] = r / N

    return gt_probs

# Apply to bigram counts
gt_bigram_probs = good_turing(bigram_counts)

print("Good-Turing Smoothed Probabilities by Frequency:")
print(f"{'Frequency':<15} {'Count':<10} {'Adjusted Prob'}")
print("=" * 40)
for freq in sorted(gt_bigram_probs.keys()):
    num_items = len([k for k, v in bigram_counts.items() if v == freq])
    print(f"{freq:<15} {num_items:<10} {gt_bigram_probs[freq]:.6f}")

Good-Turing Smoothed Probabilities by Frequency:
Frequency       Count      Adjusted Prob
1               10         0.028571
2               2          0.142857


## 5. Kneser-Ney Smoothing

**Kneser-Ney** is considered the **state-of-the-art** n-gram smoothing technique. It uses a clever insight: the probability of a word should consider how many **different contexts** it appears in, not just raw frequency.

### Key Innovation: Continuation Probability

Instead of asking "How often does 'Francisco' appear?", ask "In how many different contexts does 'Francisco' appear?"

For example:
- "San Francisco" is very common
- But "Francisco" only appears after "San"
- So "Francisco" has low **continuation probability**

**Formula (Simplified Bigram):**  
$$P_{KN}(w_2|w_1) = \frac{\max(\text{count}(w_1, w_2) - D, 0)}{\text{count}(w_1)} + \lambda_{w_1} \cdot P_{\text{continuation}}(w_2)$$

Where:
- $D$ = discount parameter (typically 0.75)
- $\lambda_{w_1}$ = normalization factor
- $P_{\text{continuation}}(w_2)$ = # of unique words before $w_2$ / total unique bigrams

**Why it's better:**
- Handles rare words more intelligently
- Considers word versatility, not just frequency
- Produces more natural language models

In [5]:
D = 0.75  # Discount parameter

def kneser_ney(unigram_counts, bigram_counts):
    """
    Apply Kneser-Ney smoothing to bigram model.
    Uses continuation probability instead of raw frequency.
    """
    # Calculate continuation counts: how many different words precede each word
    continuation_counts = defaultdict(set)
    for (w1, w2) in bigram_counts:
        continuation_counts[w2].add(w1)

    # Continuation probability: # unique contexts / total unique bigrams
    continuation_probs = {
        w: len(continuation_counts[w]) / len(bigram_counts)
        for w in continuation_counts
    }

    # Calculate Kneser-Ney probabilities
    kn_probs = {}
    for (w1, w2), count in bigram_counts.items():
        # Lambda (normalization): discount / unigram count
        lambda_w1 = D / unigram_counts[w1]
        
        # Main formula: discounted probability + continuation component
        kn_probs[(w1, w2)] = (max(count - D, 0) / unigram_counts[w1] + 
                              lambda_w1 * continuation_probs.get(w2, 0))
    
    return kn_probs

kn_bigram_probs = kneser_ney(unigram_counts, bigram_counts)

print("Kneser-Ney Probabilities:")
print(f"{'Bigram':<25} {'Count':<8} {'KN Prob':<12} {'Laplace Prob'}")
print("=" * 60)
for bigram in sorted(kn_bigram_probs.keys())[:10]:
    count = bigram_counts[bigram]
    kn_prob = kn_bigram_probs[bigram]
    lap_prob = laplace_bigram_probs.get(bigram, 0)
    print(f"{str(bigram):<25} {count:<8} {kn_prob:<12.6f} {lap_prob:.6f}")

Kneser-Ney Probabilities:
Bigram                    Count    KN Prob      Laplace Prob
('help', 'predict')       1        0.312500     0.181818
('i', 'love')             2        0.656250     0.250000
('language', 'models')    2        0.437500     0.230769
('language', 'processing') 1        0.104167     0.153846
('learning', 'language')  1        0.437500     0.181818
('love', 'learning')      1        0.156250     0.166667
('love', 'natural')       1        0.156250     0.166667
('models', 'help')        1        0.156250     0.166667
('natural', 'language')   1        0.437500     0.181818
('predict', 'words')      1        0.312500     0.181818


## 6. Sentence Probability & Perplexity

Now let's evaluate and compare our smoothing methods using two metrics:

### Sentence Probability
How likely is a given sentence according to our model?

### Perplexity
A measure of how "surprised" the model is by test data. **Lower is better.**

**Interpretation:**
- Perplexity of 10 = model is as uncertain as choosing from 10 options
- Perplexity of 100 = model is very uncertain
- Good models have low perplexity on in-domain text

In [6]:
import math

def sentence_probability(sentence, model):
    """Calculate the probability of a sentence using bigram model"""
    words = preprocess(sentence)
    prob = 1.0
    for i in range(len(words) - 1):
        prob *= model.get((words[i], words[i+1]), 1e-10)
    return prob

def perplexity(sentence, model):
    """Calculate perplexity of a sentence - lower is better"""
    words = preprocess(sentence)
    N = len(words) - 1  # Number of bigrams
    if N <= 0:
        return float('inf')
    
    log_prob = 0
    for i in range(N):
        p = model.get((words[i], words[i+1]), 1e-10)
        log_prob += math.log(p)
    
    return math.exp(-log_prob / N)

# Test sentences
test_sentences = [
    "I love language models",           # In vocabulary
    "language models help predict",     # In vocabulary
    "I enjoy learning new things"       # Partially out of vocabulary
]

print("Sentence Evaluation:\n")
for sent in test_sentences:
    print(f"Sentence: '{sent}'")
    print(f"  Kneser-Ney Probability:  {sentence_probability(sent, kn_bigram_probs):.2e}")
    print(f"  Kneser-Ney Perplexity:   {perplexity(sent, kn_bigram_probs):.2f}")
    print(f"  Laplace Probability:     {sentence_probability(sent, laplace_bigram_probs):.2e}")
    print(f"  Laplace Perplexity:      {perplexity(sent, laplace_bigram_probs):.2f}")
    print()

Sentence Evaluation:

Sentence: 'I love language models'
  Kneser-Ney Probability:  2.87e-11
  Kneser-Ney Perplexity:   3265.76
  Laplace Probability:     4.81e-03
  Laplace Perplexity:      5.92

Sentence: 'language models help predict'
  Kneser-Ney Probability:  2.14e-02
  Kneser-Ney Perplexity:   3.60
  Laplace Probability:     6.99e-03
  Laplace Perplexity:      5.23

Sentence: 'I enjoy learning new things'
  Kneser-Ney Probability:  1.00e-40
  Kneser-Ney Perplexity:   10000000000.00
  Laplace Probability:     1.00e-40
  Laplace Perplexity:      10000000000.00



## 7. Working with Real Text Data

Let's apply our models to a real book from Project Gutenberg to see how they perform at scale!

In [7]:
import urllib.request

# Download Pride and Prejudice from Project Gutenberg
url = "https://www.gutenberg.org/files/1342/1342-0.txt"

try:
    response = urllib.request.urlopen(url)
    full_text = response.read().decode('utf-8')
    
    # Remove Project Gutenberg header and footer
    start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
    end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"
    
    start_idx = full_text.find(start_marker)
    end_idx = full_text.find(end_marker)
    
    if start_idx != -1 and end_idx != -1:
        full_text = full_text[start_idx:end_idx]
    
    print(f"✓ Successfully downloaded text")
    print(f"Text length: {len(full_text):,} characters")
    print(f"\nFirst 300 characters:\n{full_text[:300]}")
    
except Exception as e:
    print(f"Error downloading: {e}")
    print("You can manually download from https://www.gutenberg.org/files/1342/1342-0.txt")
    full_text = sample_text  # Fallback to sample

✓ Successfully downloaded text
Text length: 743,334 characters

First 300 characters:
*** START OF THE PROJECT GUTENBERG EBOOK 1342 ***




                            [Illustration:

                             GEORGE ALLEN
                               PUBLISHER

                        156 CHARING CROSS ROAD
                                LONDON

                  


In [8]:
# Preprocess the full text
full_tokens = preprocess(full_text)

print(f"Total tokens: {len(full_tokens):,}")
print(f"Unique tokens: {len(set(full_tokens)):,}")
print(f"\nFirst 50 tokens: {full_tokens[:50]}")

# Build counts from full text
full_unigram_counts = Counter(full_tokens)
full_bigram_counts = defaultdict(int)

for i in range(len(full_tokens) - 1):
    full_bigram_counts[(full_tokens[i], full_tokens[i+1])] += 1

print(f"\nTotal unique unigrams: {len(full_unigram_counts):,}")
print(f"Total unique bigrams: {len(full_bigram_counts):,}")
print(f"\nMost common words: {full_unigram_counts.most_common(10)}")
print(f"\nMost common bigrams: {sorted(full_bigram_counts.items(), key=lambda x: x[1], reverse=True)[:10]}")

Total tokens: 127,166
Unique tokens: 7,201

First 50 tokens: ['start', 'of', 'the', 'project', 'gutenberg', 'ebook', 'illustration', 'george', 'allen', 'publisher', 'charing', 'cross', 'road', 'london', 'ruskin', 'house', 'illustration', 'reading', 'janes', 'letters', 'chap', 'pride', 'and', 'prejudice', 'by', 'jane', 'austen', 'with', 'a', 'preface', 'by', 'george', 'saintsbury', 'and', 'illustrations', 'by', 'hugh', 'thomson', 'illustration', 'ruskin', 'charing', 'house', 'cross', 'road', 'london', 'george', 'allen', 'chiswick', 'presscharles', 'whittingham']

Total unique unigrams: 7,201
Total unique bigrams: 57,284

Most common words: [('the', 4654), ('to', 4291), ('of', 3835), ('and', 3713), ('her', 2276), ('i', 2102), ('a', 2021), ('in', 1977), ('was', 1874), ('she', 1744)]

Most common bigrams: [(('of', 'the'), 508), (('to', 'be'), 445), (('in', 'the'), 420), (('i', 'am'), 309), (('of', 'her'), 274), (('to', 'the'), 264), (('it', 'was'), 255), (('of', 'his'), 242), (('mr', 'darc

### 7.1 Build Models on Full Text

Now let's train all three smoothing methods on the complete novel.

In [9]:
# Apply Kneser-Ney to full text (most important)
kn_full_probs = kneser_ney(full_unigram_counts, full_bigram_counts)
print(f"✓ Kneser-Ney model built: {len(kn_full_probs):,} bigram probabilities")

# Apply Good-Turing to full text
gt_full_probs = good_turing(full_bigram_counts)
print(f"✓ Good-Turing probabilities by frequency: {len(gt_full_probs)} levels")

# For comparison: sample of Laplace (computationally expensive for full vocab)
print("\n✓ All models ready for evaluation!")

✓ Kneser-Ney model built: 57,284 bigram probabilities
✓ Good-Turing probabilities by frequency: 130 levels

✓ All models ready for evaluation!


### 7.2 Compare Model Performance

Test our models on various sentences - both in-domain (from the book) and out-of-domain.

In [10]:
# Test sentences
test_sentences = [
    "it is a truth universally acknowledged",      # From the book (should score well)
    "mr darcy is a very proud man",                # From the book
    "elizabeth bennet is intelligent and witty",   # Related to book
    "I love natural language processing"           # Out of domain (should score worse)
]

print("Model Comparison on Test Sentences\n")
print(f"{'Sentence':<50} {'KN Perplexity':<15}")
print("=" * 70)

for sent in test_sentences:
    perp_kn = perplexity(sent, kn_full_probs)
    print(f"{sent:<50} {perp_kn:<15.2f}")

print("\n📊 Observations:")
print("- In-domain sentences (from the book) have lower perplexity")
print("- Out-of-domain sentences have higher perplexity")
print("- This shows the model learned the book's style!")

Model Comparison on Test Sentences

Sentence                                           KN Perplexity  
it is a truth universally acknowledged             62.01          
mr darcy is a very proud man                       28.23          
elizabeth bennet is intelligent and witty          249476768.86   
I love natural language processing                 174819981.45   

📊 Observations:
- In-domain sentences (from the book) have lower perplexity
- Out-of-domain sentences have higher perplexity
- This shows the model learned the book's style!


## 8. Extension to Trigram Models

Bigrams look at 1 previous word. **Trigrams** look at 2 previous words, capturing more context.

**Trade-off:**
- ✓ Better context understanding
- ✗ More data sparsity (need more training data)
- ✗ Higher computational cost

In [11]:
# Build trigram counts
trigram_counts = defaultdict(int)
for i in range(len(full_tokens) - 2):
    trigram_counts[(full_tokens[i], full_tokens[i+1], full_tokens[i+2])] += 1

print(f"Total trigrams: {len(trigram_counts):,}")
print(f"\nMost common trigrams:")
for trigram, count in sorted(trigram_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"  {trigram}: {count}")

Total trigrams: 107,150

Most common trigrams:
  ('i', 'do', 'not'): 66
  ('i', 'am', 'sure'): 61
  ('as', 'soon', 'as'): 56
  ('she', 'could', 'not'): 52
  ('that', 'he', 'had'): 37
  ('in', 'the', 'world'): 36
  ('copyright', 'by', 'george'): 35
  ('by', 'george', 'allen'): 35
  ('i', 'am', 'not'): 34
  ('it', 'would', 'be'): 31


In [12]:
# Trigram probabilities with Laplace smoothing (sample subset)
full_vocab = set(full_tokens)
full_V = len(full_vocab)

# Sample vocabulary for efficiency (full computation would be vocab^3)
sample_vocab = list(full_vocab)[:100]

trigram_probs = {}
for w1 in sample_vocab:
    for w2 in sample_vocab:
        for w3 in sample_vocab:
            count = trigram_counts.get((w1, w2, w3), 0)
            bigram_count = full_bigram_counts.get((w1, w2), 1)
            trigram_probs[(w1, w2, w3)] = (count + 1) / (bigram_count + full_V)

print(f"Trigram probabilities computed: {len(trigram_probs):,}")
print(f"\nSample trigram probabilities:")
for tri, prob in list(trigram_probs.items())[:5]:
    print(f"  {tri}: {prob:.8f}")

Trigram probabilities computed: 1,000,000

Sample trigram probabilities:
  ('overdone', 'overdone', 'overdone'): 0.00013885
  ('overdone', 'overdone', 'disgracing'): 0.00013885
  ('overdone', 'overdone', 'cessation'): 0.00013885
  ('overdone', 'overdone', 'settlement'): 0.00013885
  ('overdone', 'overdone', 'test'): 0.00013885


In [13]:
def trigram_perplexity(sentence, trigram_model):
    """Calculate perplexity using trigram model"""
    words = preprocess(sentence)
    N = len(words) - 2  # Number of trigrams
    if N <= 0:
        return float('inf')
    
    log_prob = 0
    for i in range(N):
        p = trigram_model.get((words[i], words[i+1], words[i+2]), 1e-10)
        log_prob += math.log(p)
    
    return math.exp(-log_prob / N)

# Compare bigram vs trigram on test sentences
print("Bigram vs Trigram Comparison\n")
print(f"{'Sentence':<50} {'Bigram PP':<12} {'Trigram PP':<12}")
print("=" * 75)

for sent in test_sentences[:3]:  # Use first 3 sentences
    perp_bi = perplexity(sent, kn_full_probs)
    perp_tri = trigram_perplexity(sent, trigram_probs)
    print(f"{sent:<50} {perp_bi:<12.2f} {perp_tri:<12.2f}")

print("\nNote: Trigrams capture more context but may have higher perplexity due to data sparsity.")

Bigram vs Trigram Comparison

Sentence                                           Bigram PP    Trigram PP  
it is a truth universally acknowledged             62.01        10000000000.00
mr darcy is a very proud man                       28.23        10000000000.00
elizabeth bennet is intelligent and witty          249476768.86 10000000000.00

Note: Trigrams capture more context but may have higher perplexity due to data sparsity.


## Key Takeaways

### Smoothing Techniques Summary

| Method | Pros | Cons | Best For |
|--------|------|------|----------|
| **Laplace** | Simple, fast | Over-smooths, wastes probability mass | Small datasets, quick prototyping |
| **Good-Turing** | Principled, frequency-aware | Complex, may need large data | Understanding frequency distributions |
| **Kneser-Ney** | State-of-the-art, context-aware | More complex implementation | Production n-gram models |

### Important Concepts

1. **Smoothing is Essential**
   - Without smoothing, unseen bigrams have 0 probability
   - This breaks sentence probability calculations
   - All practical language models use some form of smoothing

2. **Kneser-Ney is Superior**
   - Uses continuation probability, not just frequency
   - Better models natural language patterns
   - Industry standard for n-gram models

3. **Perplexity Measures Model Quality**
   - Lower perplexity = better model
   - Compare models using same test set
   - In-domain data scores better than out-of-domain

4. **Higher-Order Models Trade-Offs**
   - Trigrams > Bigrams > Unigrams for context
   - But: more parameters, more sparsity, more computation
   - Need larger training corpora

### Connection to Modern LLMs

While statistical n-gram models are foundational, modern LLMs overcome their limitations:

**Statistical LMs:**
- ✗ Limited context (2-3 words)
- ✗ Data sparsity issues
- ✗ No semantic understanding
- ✓ Fast, interpretable
- ✓ No training needed

**Neural LMs (GPT, BERT, etc.):**
- ✓ Long-range context (100s-1000s of tokens)
- ✓ Learn dense representations
- ✓ Semantic understanding
- ✗ Require training
- ✗ Less interpretable

But understanding statistical models is crucial for:
- Appreciating why neural models work so well
- Baseline comparisons
- Understanding evaluation metrics (perplexity is still used!)
- Simple, fast applications

---

✅ **You now understand advanced smoothing techniques and how to evaluate language models!**

### Next Steps
- Experiment with different discount parameters in Kneser-Ney
- Try other texts from Project Gutenberg
- Implement interpolation (combining unigram, bigram, trigram)
- Compare to neural language models!